In [1]:
import os
import json
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent, GroupChat, GroupChatManager, config_list_from_json
import autogen
import re
import time
import math
from pydantic import BaseModel, TypeAdapter
from typing import List
from autogen.agentchat import initiate_chats

/Users/alamak/miniforge3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# Define the path or environment variable for the configuration list
config_path = os.getenv("OAI_CONFIG_LIST", "OAI_CONFIG_LIST")  # Replace with the actual path

# Load configuration for the Gemini model
config_list_gemini = config_list_from_json(
    config_path,
    filter_dict={"model": ["gemini-2.0-flash"]},
)

# Load configuration for the Gemini model
config_list_gemini_json = config_list_from_json(
    config_path,
    # filter_dict={"model": ["gemini-1.5-flash"]},
    filter_dict={"usage_type": ["json"]},
)

if not config_list_gemini:
    raise ValueError("gemini-2.0-flash configuration not found. Check your config file or environment variable.")

# Initialize AssistantAgent
seed = 25  # Seed for reproducibility
llm_config={"config_list": config_list_gemini, "seed": seed}
llm_config_json={"config_list": config_list_gemini_json, "seed": seed}


In [5]:
def parse_entities_content(entities_result):
    # Handle both string input and dictionary input
    if isinstance(entities_result, dict):
        content = entities_result.get('content', '')
    elif isinstance(entities_result, str):
        content = entities_result
    else:
        raise ValueError("Invalid input type for entities_result")

    # Remove markdown code block if present
    content = re.sub(r'```json\s*|\s*```', '', content)

    # Clean up any extra whitespace
    content = content.strip()

    if not content:
        raise ValueError("The content is empty or invalid.")

    try:
        # Parse JSON string
        data = json.loads(content)

        # If `data` is a list, validate its first element
        if isinstance(data, list) and data:
            entities = data[0]
        elif isinstance(data, dict):
            entities = data
        else:
            raise ValueError("Unexpected JSON structure: not a list or dict.")

        return entities
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {str(e)}")
        print(f"Content attempting to parse: {content}")
        raise
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error accessing data structure: {str(e)}")
        raise




def remove_trailing_commas(json_string: str) -> str:
    # Regex to match trailing commas in JSON
    json_string = re.sub(r",\s*([\]}])", r"\1", json_string)  # Removes commas before } or ]
    return json_string


def get_clean_json(json_string):
    json_inspector = ConversableAgent(
        "json_inspector",
        system_message="You're a JSON interpreter. Suggest corrections to the JSON string by checking it. Also check if the contents are list or dictionary. Make sure to remove any dictionary items within a list. Say 'allgood' if it's valid JSON.",
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

    json_corrector = ConversableAgent(
        "json_corrector",
        system_message="You're a JSON corrector. Based on suggestions, you produce valid JSON only. Don't add extra text.",
        llm_config=llm_config,
        is_termination_msg=lambda msg: "allgood" in msg["content"].lower(),
        human_input_mode="NEVER",
    )

    result = json_inspector.initiate_chat(json_corrector, message=json_string, max_turns=2)
    agent_content = result.chat_history[-2]['content']

    try:
        # Use the updated parsing function
        return parse_entities_content({"content": agent_content})
    except Exception as e:
        print(f"Failed to parse entities: {str(e)}")
        raise


def get_entities(agent, content):
    message = [{"content": content, "role": "user"}]
    agent_result = agent.generate_reply(messages=message)

    try:
        print("Trying to parse for 1st time...")
        result = parse_entities_content(agent_result)
        print("Successfully parsed items:", result)
        return result
    except Exception as e:
        print(f"Failed to parse items: {str(e)}")
        print("Trying parsing for 2nd time...")
        json_string = re.sub(r'```json\s*|\s*```', '', agent_result['content']).strip()
        return get_clean_json(json_string)


In [ ]:

# Define the schema and user input
questions_schema = [
    {
        "questions": [
            "Question 1 goes here",
            "Question 2 goes here",
            "Question 3 goes here",
            "..."
        ]
    }
]
qs_agent = ConversableAgent(
    "questions",
    system_message=f"""You are tasked with generating review questions in json structured outlined below to help learners deeply understand a given topic.

Consider the topic from multiple perspectives based on the provided transcript.

Expected output: 
{questions_schema}
""",
    llm_config=llm_config_json,
    human_input_mode="NEVER",  # Never ask for human input.
)

